In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [5]:
from pyspark.context import SparkContext 
sc = SparkContext.getOrCreate()
from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate('local')
spark = SparkSession(sc)

In [6]:
import os
print os.getcwd()


/Users/bharvadl/Desktop/CourseWork/Search/project


In [7]:
import pandas as pd

In [8]:
##Toy Dataset code

df_rev = pd.read_csv("review.csv")

# lines = spark.read.csv(mpath+"review.csv").rdd
# parts = lines.map(lambda row: row.value.split("::"))

In [9]:
print df_rev
df_rev = df_rev.sample(frac=0.4)
ratings = spark.createDataFrame(df_rev)

         funny                 user_id               review_id  \
0            0  cjpdDjZyprfyDG3RlkVG3w  VfBHSwC5Vz_pbFluy07i9Q   
1            0  bjTcT8Ty4cJZhEOEo01FGA  3zRpneRKDsOPq92tq7ybAA   
2            0  AXgRULmWcME7J6Ix3I--ww  ne5WhI1jUFOcRn-b-gAzHA   
3            0  oU2SSOmsp_A8JYI7Z2JJ5w  llmdwOgDReucVoWEry61Lw   
4            0  0xtbPEna2Kei11vsU-U2Mw  DuffS87NaSMDmIfluvT83g   
5            0  rW8q706dz5-NnXDzMwVkiw  GvLmUkjUrOyFH8KFnmT1uw   
6            0  yx8vNXUL0D0HS8rUIC7AFA  lGEl24NGj2HVBJrodeXcjg   
7            0  zXnH6W74FAJQ7q7b-NuBsA  cUgvEy5wj7zYE68v1BzzVg   
8            0  c5yp5hxwC1N98MjbV2LyWQ  FSB_BnvysBgH3JYrbFNcgw   
9            0  xJisL5w4wOgiYLokGMT_IA  dhl3ZW9aAEX_T7_um5tfaQ   
10           1  tgV6tsYQ66DZ3LQKvtC6cw  JQJvnM3p-3eML05eKcTgiw   
11           0  Q-3YCVywc03w56wYtGlKvg  6JF4WfHgwYrrdZ2VeYtnFw   
12           0  Cx4UCow0zQgFQOp47RRRaA  fbVYETRuWDw8QnpimpGMpg   
13           0  eqWEgMH-DCP74i82BEAZzw  lobj38NgaokqVseN8_nn6g   
14        

In [15]:
df_new = df_rev[['user_id','business_id','stars']]
df_new = pd.DataFrame(df_new.groupby(['user_id','business_id']).agg('mean'))
df_new.reset_index(inplace=True)
df_new.head()

,user_id,business_id,stars
0,---1lKK3aKOuomHnwAkAow,1Vn_lex3LGGwuTo-xeJnww,5
1,---1lKK3aKOuomHnwAkAow,DV13F0bhe55dV1AhwoO50g,5
2,---1lKK3aKOuomHnwAkAow,HZdtHOEaKUL2SlWj5owgCA,5
3,---1lKK3aKOuomHnwAkAow,Ks0M3J4vZAKsHPuCINz5fQ,4
4,---1lKK3aKOuomHnwAkAow,NAuUGBenC_TJoj2c6y9rbg,5


In [19]:
df_new['user_id_int'] = xrange(df_new.shape[0])

df_new['business_id_int'] = xrange(df_new.shape[0])

In [29]:
ratings = spark.createDataFrame(df_new)
print df_new,ratings

                       user_id             business_id  stars  user_id_int  \
0       ---1lKK3aKOuomHnwAkAow  1Vn_lex3LGGwuTo-xeJnww      5            0   
1       ---1lKK3aKOuomHnwAkAow  DV13F0bhe55dV1AhwoO50g      5            1   
2       ---1lKK3aKOuomHnwAkAow  HZdtHOEaKUL2SlWj5owgCA      5            2   
3       ---1lKK3aKOuomHnwAkAow  Ks0M3J4vZAKsHPuCINz5fQ      4            3   
4       ---1lKK3aKOuomHnwAkAow  NAuUGBenC_TJoj2c6y9rbg      5            4   
5       ---1lKK3aKOuomHnwAkAow  PT9BRvRWx9kpcu02VouW1g      2            5   
6       ---1lKK3aKOuomHnwAkAow  R-McIj4Psxl1VlEacsXeRg      4            6   
7       ---1lKK3aKOuomHnwAkAow  RhTBGAHFqnFTgSUDJtBuIQ      5            7   
8       ---1lKK3aKOuomHnwAkAow  eduRavkml8awmPachSZXuw      1            8   
9       ---1lKK3aKOuomHnwAkAow  jGv6tpmE7F9CG5qmLHad6A      5            9   
10      ---1lKK3aKOuomHnwAkAow  jobP3ywRd3QNZ_GCoPG2DQ      3           10   
11      ---1lKK3aKOuomHnwAkAow  nv2z5SYQR0JlzflUrm00Mw      5   

In [32]:

# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
#                                      rating=float(p[2]), timestamp=long(p[3])))
# ratings = spark.createDataFrame(ratingsRDD)

# (training, test) = ratings.randomSplit([0.8, 0.2])

# # Build the recommendation model using ALS on the training data
# # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# als = ALS(maxIter=5, regParam=0.01, userCol="user_id_int", itemCol="business_id_int", ratingCol="stars")
# model = als.fit(training)

# # Evaluate the model by computing the RMSE on the test data
# predictions = model.transform(test)
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars",
#                                 predictionCol="prediction")
# print predictions
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

# # # Generate top 10 movie recommendations for each user
# # userRecs = model.recommendForAllUsers(10)
# # # Generate top 10 user recommendations for each movie
# # movieRecs = model.recommendForAllItems(10)

DataFrame[user_id: string, business_id: string, stars: bigint, user_id_int: bigint, business_id_int: bigint]                        user_id             business_id  stars  user_id_int  \
0       ---1lKK3aKOuomHnwAkAow  1Vn_lex3LGGwuTo-xeJnww      5            0   
1       ---1lKK3aKOuomHnwAkAow  DV13F0bhe55dV1AhwoO50g      5            1   
2       ---1lKK3aKOuomHnwAkAow  HZdtHOEaKUL2SlWj5owgCA      5            2   
3       ---1lKK3aKOuomHnwAkAow  Ks0M3J4vZAKsHPuCINz5fQ      4            3   
4       ---1lKK3aKOuomHnwAkAow  NAuUGBenC_TJoj2c6y9rbg      5            4   
5       ---1lKK3aKOuomHnwAkAow  PT9BRvRWx9kpcu02VouW1g      2            5   
6       ---1lKK3aKOuomHnwAkAow  R-McIj4Psxl1VlEacsXeRg      4            6   
7       ---1lKK3aKOuomHnwAkAow  RhTBGAHFqnFTgSUDJtBuIQ      5            7   
8       ---1lKK3aKOuomHnwAkAow  eduRavkml8awmPachSZXuw      1            8   
9       ---1lKK3aKOuomHnwAkAow  jGv6tpmE7F9CG5qmLHad6A      5            9   
10      ---1lKK3aKOuomHnwAkAow  j